![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [1]:
# Make sure to run this cell to use torchmetrics. If you cannot use pip install to install the torchmetrics, you can use sklearn.
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 38.2 MB/s eta 0:00:00


In [2]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchmetrics import Accuracy
# from sklearn.metrics import accuracy_score  # uncomment to use sklearn

In [3]:
# Load preprocessed data
train_df = pd.read_csv('labelled_train.csv')
test_df = pd.read_csv('labelled_test.csv')
val_df = pd.read_csv('labelled_validation.csv')

# View the first 5 rows of training set
train_df.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue,sus_label
0,381,7337,1,100,4026532231,5,0,1
1,381,7337,1,100,4026532231,1,0,1
2,381,7337,1,100,4026532231,0,0,1
3,7347,7347,7341,0,4026531840,2,-2,1
4,7347,7347,7341,0,4026531840,4,0,1


In [4]:
#separating features
y_train=train_df["sus_label"]
x_train=train_df.iloc[:,:-1]
y_test=test_df["sus_label"]
x_test=test_df.iloc[:,:-1]
y_val=val_df["sus_label"]
x_val=val_df.iloc[:,:-1]

In [5]:
#scaling
scaler=StandardScaler()
scaler.fit(x_train)
scaled_x_train=scaler.transform(x_train)
scaled_x_test=scaler.transform(x_test)
scaled_x_val=scaler.transform(x_val)


In [6]:
#convert_to_pytorch

x_train=torch.tensor(scaled_x_train).to(dtype=torch.float32)
y_train=torch.tensor(y_train)
x_test=torch.tensor(scaled_x_test).to(dtype=torch.float32)
y_test=torch.tensor(y_test)
x_val=torch.tensor(scaled_x_val).to(dtype=torch.float32)
y_val=torch.tensor(y_val)
print(y_train.shape)
train_dataset=TensorDataset(x_train,y_train)
test_dataset=TensorDataset(x_test,y_test)
val_dataset=TensorDataset(x_val,y_val)

torch.Size([763144])


In [7]:
import torch.nn as nn
import torch.optim as optim

# model
model = nn.Sequential(
    nn.Linear(7, 5),
    nn.ReLU(),
    nn.Linear(5, 3),
    nn.ReLU(),
    nn.Linear(3, 2),
    nn.Sigmoid()
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.93)

In [8]:
#trainloop
num_epochs=3
batch_size=32
data_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
for epoch in range(num_epochs):
    model.train()
    print("done")
    for inputs, targets in data_loader:
        inputs=inputs.to(dtype=torch.float32)
        targets=targets.to(dtype=torch.int64)
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.double(), targets)
        # Backward pass
        loss.backward() 
        # Update the parameters
        optimizer.step()

    


done
done
done


In [13]:
import torch
import torchmetrics
from torch.utils.data import DataLoader

accuracy_metric = torchmetrics.Accuracy(task='binary', num_classes=2)
model.eval()
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

with torch.no_grad():
    for batch_features, batch_targets in val_loader:
        batch_features = batch_features.to(dtype=torch.float32)
        batch_targets = batch_targets.to(dtype=torch.int64)
        
        outputs = model(batch_features)
        predictions = outputs.argmax(dim=-1)
        # Update accuracy metric
        accuracy_metric.update(predictions, batch_targets)

accuracy = accuracy_metric.compute()
print(f'Validation Accuracy: {accuracy:.4f}')

Validation Accuracy: 0.9958
